In [76]:
import pandas as pd
import numpy as np
from IPython import display
from pymongo import MongoClient
import os
import re
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

display.clear_output(wait=True)
from config import user_name,password,ip
from epub_conversion.utils import open_book

In [73]:
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
stop = set(stop)

In [2]:
configString = "mongodb://"+user_name+":"+password+"@"+ip+"/fletcher"

In [3]:
client = MongoClient(configString) # defaults to port 27017

db = client.fletcher

# print the number of documents in a collection
#print (db.collection_names())

In [36]:
#print(db.create_collection('books'))

Collection(Database(MongoClient(host=['34.207.110.104:27017'], document_class=dict, tz_aware=False, connect=True), 'fletcher'), 'books')


In [15]:
db.collection_names()

['books', 'test']

In [38]:
!pwd

/Users/xavier/dev/metis/fletcher


**Create a dictionary from all books**

In [16]:
book_list = []
book_dict = {}
path = "/Users/xavier/dev/metis/fletcher/books/"
for file in os.listdir(path):
    if file.endswith(".txt"):
        clean_name = file.replace(" - Stephen King.txt","")
        book_dict[clean_name] = open(path+file, "r").read()
        book_list.append(file)

In [56]:
# Insert the books into mongo db
for k,v in book_dict.items():
    try:
        year = re.search("[Cc]opyright ©\s*.*_*(\d{4}).*Stephen King|[Cc]opyright ©\s.*Stephen King.*_*(\d{4})|[Cc]opyright ©\s*.*_*(\d{4}).*Richard Bachman|[Cc]opyright ©\s.*Richard Bachman.*_*(\d{4})",v).group(0) # get copyright year from book text
        year = re.search("(\d{4})",year).group(0)
    except:
        year = ""
    try:
        isbn = re.search(".*ISBN+:*(\d*.*)",v)[1].split(" ")
        isbn = max(isbn, key=len)
    except:
        isbn = ""
        
    doc = {"title":k,"text":v,"year":year,'isbn':isbn}
    #print(doc['title'],doc['isbn'])
    #print(doc['title'],doc['year'])
    db.books.insert_one(doc)

Throttle 9780062215956
Night Shift 978-0-385-52884-9
Riding the Bullet 0-7432-0467-0
Roadwork 978-1-101-13813-7
Salem's Lot 978-0-385-52822-1
Misery 978-1-101-13812-0
Everything's Eventual_ 14 Dark Tales 978-1-4165-4985-7
Song of Susannah 978-1-4165-2149-5
Wizard and Glass 1-101-14644-3
Thinner 978-1-101-13805-2
Hearts in Atlantis 0-671-02424-8
Dolores Claiborne 978-1-101-13817-5
Finders Keepers 978-1-5011-0007-9
Insomnia 978-1-101-13800-7
Different Seasons 978-1-101-13808-3
Dead Zone, The 978-1-101-13814-4
Bag of Bones_ A Novel 978-1-4391-0621-1
Tommyknockers, The 978-1-101-13804-5
Wind Through the Keyhole_ A Dark Tower Novel, The 978-1-4516-5890-3
Mist, The 978-1-1012-1166-3
Gerald's Game 978-1-101-13815-1
Nightmares & Dreamscapes 978-1-4391-0256-5
Big Driver 978-1-5011-0443-5
Good Marriage, A 978-1-5011-0442-8
From a Buick 8_ A Novel 0-7432-1137-5
Revival 978-1-4767-7038-3
Eyes of the Dragon, The 978-1-101-13807-6
Waste Lands, The 978-1-101-15861-6
Gunslinger, The 1-101-14645-1
Girl

**Now that the books are in mongo lets try to play with the text data**

In [48]:
# db.books.remove({})

In [52]:
carrie = db.books.find({'title':'Carrie'}).next()

In [55]:
carrie['isbn']

'978-0-385-52883-2'

In [61]:
len(carrie['text'])

340413

In [66]:
#convert all text to lowercase
c_text = carrie['text'].lower()
c_text = c_text.strip('\n')

In [68]:
carrie = TextBlob(c_text)

In [70]:
carrie.tags

[('carrie', 'NN'),
 ('contents', 'NNS'),
 ('cover', 'VBP'),
 ('page', 'NN'),
 ('title', 'JJ'),
 ('page', 'NN'),
 ('dedication', 'NN'),
 ('part', 'NN'),
 ('one', 'CD'),
 ('blood', 'NN'),
 ('sport', 'NN'),
 ('news', 'NN'),
 ('item', 'NN'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('westover', 'NN'),
 ('part', 'NN'),
 ('two', 'CD'),
 ('prom', 'NN'),
 ('night', 'NN'),
 ('she', 'PRP'),
 ('put', 'VBD'),
 ('the', 'DT'),
 ('dress', 'NN'),
 ('on', 'IN'),
 ('for', 'IN'),
 ('the', 'DT'),
 ('first', 'JJ'),
 ('part', 'NN'),
 ('three', 'CD'),
 ('wreckage', 'NN'),
 ('westover', 'WRB'),
 ('mercy', 'NN'),
 ('hospital/report', 'NN'),
 ('of', 'IN'),
 ('decease', 'NN'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('national', 'JJ'),
 ('ap', 'NN'),
 ('ticker', 'NN'),
 ('by', 'IN'),
 ('stephen', 'JJ'),
 ('king', 'VBG'),
 ('copyright', 'NN'),
 ('this', 'DT'),
 ('is', 'VBZ'),
 ('for', 'IN'),
 ('tabby', 'NN'),
 ('who', 'WP'),
 ('got', 'VBD'),
 ('me', 'PRP'),
 ('into', 'IN'),
 ('it—', 'NN'),
 ('and', 'CC'),
 ('then', 'RB'),
 ('

In [85]:
words = [''.join(words) for words in c_text.split()]
# vectorizer = TfidfVectorizer(stop_words=stop, ngram_range=(1))
# doc_vectors = vectorizer.fit_transform(documents)

In [88]:
len(words)

61289

AttributeError: 'str' object has no attribute 'generate'